# Step 2: Feature Engineering

Feature engineering combines the different data sources together to create a single data set of features (variables) that can be used to infer a machines's health condition over time. 

In this notebook, we will load the data stored in Azure Blob containers in the previous **Data Ingestion** notebook (`Code/data_ingestion.ipynb`). The note book uses several feature engineering methods to create a data set for use in our predictive maintenance machine learning solution.

In [43]:
# import the libraries
import os
import glob
import pandas as pd 

import pyspark.sql.functions as F
from pyspark.sql.functions import col, unix_timestamp, round
from pyspark.sql.functions import datediff
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType

from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.feature import StringIndexer

from pyspark.sql import SparkSession

from azure.storage.blob import BlockBlobService
from azure.storage.blob import PublicAccess

spark = SparkSession.builder.getOrCreate()

## Load data from Azure Blob storage container

We have previously downloaded and stored the following data in an Azure blob storage container:

  * Machines: Features differentiating each machine. For example age and model.
  * Error: The log of non-critical errors. These errors may still indicate an impending component failure.
  * Maint: Machine maintenance history detailing component replacement or regular maintenance activities withe the date of replacement.
  * Telemetry: The operating conditions of a machine e.g. data collected from sensors.
  * Failure history: The failure history of a machine or component within the machine.

We'll load these files from blob, and create our analysis data set here. We'll write this data set back into a new blob container to use in our model building and evaluation notebook later. 

Since the Azure Blob storage account name and account key are not passed between notebooks, you'll need to provide those here again.

In [44]:
# Enter your Azure blob storage details here 
ACCOUNT_NAME = "pdmamlworkbench"   ## "<your blob storage account name>"

# You can find the account key under the _Access Keys_ link in the 
# [Azure Portal](portal.azure.com) page for your Azure storage container.
ACCOUNT_KEY = "O5uLzNKX7o+ZHFXtHDyS87SIev9QHlkdX2IhIbxYwhRo7sA9zp45HOOFFttUp4r0LyWCcLQ0cCA7l+e8Ct3Yew==" ## "<account key>"

#-------------------------------------------------------------------------------------------
# The data from the Data Aquisition note book is stored in the dataingestion container.
CONTAINER_NAME = "dataingestion"

# The data constructed in this notebook will be stored in the featureengineering container
STORAGE_CONTAINER_NAME = "featureengineering"

# Connect to your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

# We will store each of these data sets in blob storage in an 
# Azure Storage Container on your Azure subscription.
# See https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md
# for details.

# These file names detail which blob each file is stored under. 
MACH_DATA = 'machines_files.parquet'
MAINT_DATA = 'maint_files.parquet'
ERROR_DATA = 'errors_files.parquet'
TELEMETRY_DATA = 'telemetry_files.parquet'
FAILURE_DATA = 'failure_files.parquet'

# These file names detail the local paths where we store the data results.
MACH_LOCAL_DIRECT = 'dataingestion_mach_result.parquet'
ERROR_LOCAL_DIRECT = 'dataingestion_err_result.parquet'
MAINT_LOCAL_DIRECT = 'dataingestion_maint_result.parquet'
TELEMETRY_LOCAL_DIRECT = 'dataingestion_tel_result.parquet'
FAILURES_LOCAL_DIRECT = 'dataingestion_fail_result.parquet'

# This is the final data file.
FEATURES_LOCAL_DIRECT = 'featureengineering_files.parquet'

# define your blob service     
my_service = BlockBlobService(account_name=ACCOUNT_NAME, account_key=ACCOUNT_KEY)

### Machines data set

Load the machines data set from your Azure blob.

In [45]:
# download the entire parquet result folder to local path for a new run 

# create a local path  to store the data.
if not os.path.exists(MACH_LOCAL_DIRECT):
    os.makedirs(MACH_LOCAL_DIRECT)
    print('DONE creating a local directory!')

# Connect to blob storage container
for blob in my_service.list_blobs(CONTAINER_NAME):
    if MACH_DATA in blob.name:
        local_file = os.path.join(MACH_LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

# Read in the data
machines = spark.read.parquet(MACH_LOCAL_DIRECT)

print(machines.count())
machines.limit(5).toPandas().head()

1000


,machineID,model,age
0,1,model2,18
1,2,model4,7
2,3,model3,8
3,4,model3,7
4,5,model2,2


### Errors data set

Load the errors data set from your Azure blob.

In [46]:
# load the previous created final dataset into the workspace

# create a local path  to store the data.
if not os.path.exists(ERROR_LOCAL_DIRECT):
    os.makedirs(ERROR_LOCAL_DIRECT)
    print('DONE creating a local directory!')

# Connect to blob storage container
for blob in my_service.list_blobs(CONTAINER_NAME):
    if ERROR_DATA in blob.name:
        local_file = os.path.join(ERROR_LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

# Read in the data
errors = spark.read.parquet(ERROR_LOCAL_DIRECT)

print(errors.count())
errors.limit(5).toPandas().head()

11967


,datetime,machineID,errorID
0,2015-04-08 19:00:00,251,error3
1,2015-06-09 06:00:00,251,error1
2,2015-08-08 06:00:00,251,error4
3,2015-09-07 06:00:00,251,error2
4,2015-09-07 06:00:00,251,error3


### Maintenance data set

Load the maintenance data set from your Azure blob.

In [47]:
# load the previous created final dataset into the workspace

# create a local path  to store the data.
if not os.path.exists(MAINT_LOCAL_DIRECT):
    os.makedirs(MAINT_LOCAL_DIRECT)
    print('DONE creating a local directory!')

# Connect to blob storage container
for blob in my_service.list_blobs(CONTAINER_NAME):
    if MAINT_DATA in blob.name:
        local_file = os.path.join(MAINT_LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

# Read in the data
maint = spark.read.parquet(MAINT_LOCAL_DIRECT)

print(maint.count())
maint.limit(5).toPandas().head()

32592


,datetime,machineID,comp
0,2015-01-04 06:00:00,252,comp1
1,2015-01-19 06:00:00,252,comp4
2,2015-02-18 06:00:00,252,comp3
3,2015-03-05 06:00:00,252,comp2
4,2015-03-20 06:00:00,252,comp1


### Telemetry

Load the telemetry data set from your Azure blob.

In [48]:
# load the previous created final dataset into the workspace

# create a local path  to store the data.
if not os.path.exists(TELEMETRY_LOCAL_DIRECT):
    os.makedirs(TELEMETRY_LOCAL_DIRECT)
    print('DONE creating a local directory!')

# Connect to blob storage container
for blob in my_service.list_blobs(CONTAINER_NAME):
    if TELEMETRY_DATA in blob.name:
        local_file = os.path.join(TELEMETRY_LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

# Read in the data
telemetry = spark.read.parquet(TELEMETRY_LOCAL_DIRECT)

print(telemetry.count())
telemetry.limit(5).toPandas().head()

8761000


,datetime,machineID,volt,rotate,pressure,vibration
0,2015-01-08 10:00:00,501,165.775142,456.014484,96.779707,40.200315
1,2015-01-08 11:00:00,501,167.694494,415.396525,106.346838,39.454320
2,2015-01-08 12:00:00,501,149.286911,549.794168,110.590462,46.649346
3,2015-01-08 13:00:00,501,164.315444,485.343432,102.644426,38.615502
4,2015-01-08 14:00:00,501,178.789891,447.830204,100.238279,36.380291


### Failures data set

Load the failures data set from your Azure blob.

In [49]:
# load the previous created final dataset into the workspace

# create a local path  to store the data.
if not os.path.exists(FAILURES_LOCAL_DIRECT):
    os.makedirs(FAILURES_LOCAL_DIRECT)
    print('DONE creating a local directory!')


# download the entire parquet result folder to local path for a new run 
for blob in my_service.list_blobs(CONTAINER_NAME):
    if FAILURE_DATA in blob.name:
        local_file = os.path.join(FAILURES_LOCAL_DIRECT, os.path.basename(blob.name))
        my_service.get_blob_to_path(CONTAINER_NAME, blob.name, local_file)

failures = spark.read.parquet(FAILURES_LOCAL_DIRECT)

print(failures.count())
failures.limit(5).toPandas().head()

6726


,datetime,machineID,failure
0,2015-09-18 06:00:00,453,comp2
1,2015-12-17 06:00:00,453,comp2
2,2015-03-27 06:00:00,454,comp2
3,2015-08-24 06:00:00,454,comp2
4,2015-09-23 06:00:00,454,comp1


## Feature engineering 

Feature engineering combines the different data sources together to create a single data set of features (variables) that can be used to infer a machines's health condition over time. The ultimate goal is to generate a single record for each time unit for each asset combining its features and labels to be fed into the machine learning algorithm. In order to prepare that clean final data set, some pre-processing steps should be taken. First step is to divide the duration of data collection into time units where each record belongs to a time unit for an asset.

The measurement unit for time can be in seconds, minutes, hours, days, months, cycles, miles or transactions depending on the efficiency of data preparation and the changes observed in the conditions of the asset from a time unit to the other or other factors specific to the domain. In other words, the time unit does not have to be the same as the frequency of data collection as in many cases data may not show any difference from one unit to the other. For example, if temperature values were being collected every 10 seconds, picking a time unit of 10 seconds for the whole analysis inflates the number of examples without providing any additional information. Better strategy would be to use average over an hour as an example.

### Rolling aggregates

For each record of an asset, we pick a rolling window of size "W" which is the number of units of time that we would like to compute historical aggregates for. We then compute rolling aggregate features using the W periods before the date of that record. Some example rolling aggregates can be rolling counts, means, standard deviations, outliers based on standard deviations, CUSUM measures, minimum and maximum values for the window. Another interesting technique is to capture trend changes, spikes and level changes using algorithms that detect anomalies in data using anomaly detection algorithms.

### Lag features
As mentioned earlier, in predictive maintenance, historical data usually comes with timestamps indicating the time of collection for each piece of data. There are many ways of creating features from the data that comes with timestamped data. In this section, we discuss some of these methods used for predictive maintenance. However, we are not limited by these methods alone. Since feature engineering is considered to be one of the most creative areas of predictive modeling, there could be many other ways to create features. Here, we provide some general techniques.

## Telemetry features

Because the telemetry data set is the largest time series data we have, we start feature engineering here. 

A common method is to pick a window size for the lag features to be created and compute rolling aggregate measures such as mean, standard deviation, minimum, maximum, etc. to represent the short term history of the telemetry over the window. In the following, rolling mean and standard deviation of the telemetry data over the last 3 hour and 24 hour lag windows is calculated for every 3 hours.



In [50]:
# rolling mean
# Temporary storage for rolling means
tel_mean = telemetry

# Which features are we interested in telemetry data set
rolling_features = ['volt','rotate', 'pressure', 'vibration']
               
# We choose two windows for our rolling windows 3hrs, 24 hrs
lags = [3,24]

for lag_n in lags:
    wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        tel_mean = tel_mean.withColumn(col_name+'_rollingmean_'+str(lag_n), F.avg(col(col_name)).over(wSpec))

# 3 hours = 10800 seconds  
time_val = 3 * 60 * 60

# I think this is grabbing datetime from the tel_sd data set, which is equivalent to telemetry
dt_truncated = ((round(unix_timestamp(col("datetime")) / time_val) * time_val).cast("timestamp"))

tel_mean_resampled = tel_mean.withColumn("dt_truncated", dt_truncated).drop('volt', 'rotate', 'pressure', 'vibration')

tel_mean = (tel_mean_resampled.groupBy("machineID","dt_truncated")
            .agg(F.mean('volt_rollingmean_3').alias('volt_rollingmean_3'),
                 F.mean('rotate_rollingmean_3').alias('rotate_rollingmean_3'), 
                 F.mean('pressure_rollingmean_3').alias('pressure_rollingmean_3'), 
                 F.mean('vibration_rollingmean_3').alias('vibration_rollingmean_3'), 
                 F.mean('volt_rollingmean_24').alias('volt_rollingmean_24'),
                 F.mean('rotate_rollingmean_24').alias('rotate_rollingmean_24'), 
                 F.mean('pressure_rollingmean_24').alias('pressure_rollingmean_24'), 
                 F.mean('vibration_rollingmean_24').alias('vibration_rollingmean_24')))

# Clean up temp memory space
tel_mean_resampled.unpersist()

# rolling standard deviation
# Temporary storage for rolling means
tel_sd = telemetry

for lag_n in lags:
    wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-lag_n, 0)
    for col_name in rolling_features:
        tel_sd = tel_sd.withColumn(col_name+'_rollingstd_'+str(lag_n), F.stddev(col(col_name)).over(wSpec))
    
# tel_sd rolling sd
tel_sd_resampled = (tel_sd.withColumn("dt_truncated", dt_truncated)
                    .drop('volt', 'rotate', 'pressure', 'vibration')
                    .fillna(0))

tel_sd = (tel_sd_resampled.groupBy("machineID","dt_truncated")
          .agg(F.stddev('volt_rollingstd_3').alias('volt_rollingstd_3'),
               F.stddev('rotate_rollingstd_3').alias('rotate_rollingstd_3'), 
               F.stddev('pressure_rollingstd_3').alias('pressure_rollingstd_3'), 
               F.stddev('vibration_rollingstd_3').alias('vibration_rollingstd_3'), 
               F.stddev('volt_rollingstd_24').alias('volt_rollingstd_24'),
               F.stddev('rotate_rollingstd_24').alias('rotate_rollingstd_24'), 
               F.stddev('pressure_rollingstd_24').alias('pressure_rollingstd_24'), 
               F.stddev('vibration_rollingstd_24').alias('vibration_rollingstd_24')))

# clean temp storage
tel_sd_resampled.unpersist()

telemetry_feat = (tel_mean.join(tel_sd,
                                ((tel_mean['machineID'] == tel_sd['machineID']) 
                                 & (tel_mean['dt_truncated'] == tel_sd['dt_truncated'])), 
                                "left")
                  .drop(tel_sd.machineID).drop(tel_sd.dt_truncated))

telemetry_feat.where((col("machineID") == 1)).limit(10).toPandas().head(10)

,machineID,dt_truncated,volt_rollingmean_3,rotate_rollingmean_3,pressure_rollingmean_3,vibration_rollingmean_3,volt_rollingmean_24,rotate_rollingmean_24,pressure_rollingmean_24,vibration_rollingmean_24,volt_rollingstd_3,rotate_rollingstd_3,pressure_rollingstd_3,vibration_rollingstd_3,volt_rollingstd_24,rotate_rollingstd_24,pressure_rollingstd_24,vibration_rollingstd_24
0,1,2015-01-09 06:00:00,161.758220,460.191228,96.335812,39.051541,168.100278,438.408323,100.435705,41.143014,2.542234,13.617558,1.636829,1.264601,0.390377,0.696726,0.390516,0.265077
1,1,2015-04-02 12:00:00,177.068429,433.378560,96.498988,38.821066,172.384126,447.099119,100.856740,39.575157,5.793824,12.301515,1.270456,0.217692,0.018506,0.784804,0.176249,0.034673
2,1,2015-04-11 18:00:00,171.014696,411.497993,97.440480,41.011138,172.095467,443.324774,98.037775,39.499885,6.437136,16.375696,3.145606,0.463779,0.219697,2.096753,0.296434,0.096999
3,1,2015-04-17 15:00:00,161.944471,429.707358,100.453624,43.356965,166.049779,442.574675,98.379748,41.186337,10.772187,1.933757,2.839066,3.337395,0.530808,0.569094,0.034361,0.037792
4,1,2015-04-18 21:00:00,178.593778,457.468729,101.436350,43.940496,172.648494,447.924559,99.065954,40.058577,4.205957,19.861217,4.228484,3.509604,0.304786,2.469812,0.436034,0.157498
5,1,2015-04-22 18:00:00,173.814460,481.608946,106.691878,38.773940,170.510248,458.411583,101.900838,41.187209,5.185854,23.642292,1.652761,0.906791,0.530828,3.221165,0.529525,0.025674
6,1,2015-05-12 00:00:00,177.166806,431.077395,98.427573,32.993633,171.314120,456.254551,101.924653,40.758712,11.157716,8.538429,1.468329,1.573847,0.624328,0.452415,0.265851,0.314688
7,1,2015-06-02 03:00:00,169.390774,421.496917,111.307572,41.276803,165.348173,447.698849,100.007553,39.866762,0.479267,11.691539,1.553642,0.801571,0.222697,1.168570,0.287293,0.025985
8,1,2015-07-26 12:00:00,165.344056,442.703231,87.036960,40.620244,168.613437,445.626234,96.981386,39.743980,5.748553,11.023769,4.968860,2.466889,0.785477,0.683333,0.425018,0.094826
9,1,2015-08-04 00:00:00,165.434594,492.411368,102.873017,39.090342,171.040596,474.751091,102.689331,42.860270,3.807173,3.990032,3.853427,0.570134,0.162140,1.228408,0.353189,0.056700


## Errors features

Like telemetry data, errors come with timestamps. An important difference is that the error IDs are categorical values and should not be averaged over time intervals like the telemetry measurements. Instead, we count the number of errors of each type in a lagging window. We begin by reformatting the error data to have one entry per machine per time at which at least one error occurred.

In [51]:
# create a column for each errorID 
error1 = errors.groupBy("machineID","datetime","errorID").pivot('errorID').agg(F.count('machineID').alias('dummy'))

# remove the column called errorID and fill in missing values
error2 = error1.drop('errorID').fillna(0)

# combine errors for a given machine in a given hour
error3 = (error2.groupBy("machineID","datetime")
                .agg(F.sum('error1').alias('error1sum'), 
                     F.sum('error2').alias('error2sum'), 
                     F.sum('error3').alias('error3sum'), 
                     F.sum('error4').alias('error4sum'), 
                     F.sum('error5').alias('error5sum')))

# join the telemetry data with errors
error_count = (telemetry.join(error3, ((telemetry['machineID'] == error3['machineID']) 
                                       & (telemetry['datetime'] == error3['datetime'])), "left")
               .drop('volt', 'rotate', 'pressure', 'vibration').drop(error3.machineID).drop(error3.datetime))

# fill in missing value
err_mean = error_count.fillna(0)

# remove 
error_count.unpersist()
error1.unpersist()
error2.unpersist()
error3.unpersist()

error_features = ['error1sum','error2sum', 'error3sum', 'error4sum', 'error5sum']

wSpec = Window.partitionBy('machineID').orderBy('datetime').rowsBetween(1-24, 0)
for col_name in error_features:
    err_mean = err_mean.withColumn(col_name+'_rollingmean_'+str(lag_n), F.avg(col(col_name)).over(wSpec))

dt_truncated = ((round(unix_timestamp(col("datetime").cast("timestamp")) / time_val) * time_val)
    .cast("timestamp"))

err_mean_resampled = (err_mean.withColumn("dt_truncated", dt_truncated)
                    .drop('error1sum', 'error2sum', 'error3sum', 'error4sum', 'error5sum').fillna(0))

err_feat = (err_mean_resampled.groupBy("machineID","dt_truncated")
            .agg(F.mean('error1sum_rollingmean_24').alias('error1sum_rollingmean_24'), 
                 F.mean('error2sum_rollingmean_24').alias('error2sum_rollingmean_24'), 
                 F.mean('error3sum_rollingmean_24').alias('error3sum_rollingmean_24'), 
                 F.mean('error4sum_rollingmean_24').alias('error4sum_rollingmean_24'), 
                 F.mean('error5sum_rollingmean_24').alias('error5sum_rollingmean_24')))

err_mean_resampled.unpersist()

err_feat.count(), len(err_mean.columns)
err_feat.limit(10).toPandas().head(10)

,machineID,dt_truncated,error1sum_rollingmean_24,error2sum_rollingmean_24,error3sum_rollingmean_24,error4sum_rollingmean_24,error5sum_rollingmean_24
0,29,2015-01-26 03:00:00,0.0,0.0,0.0,0.0,0.0
1,29,2015-01-29 21:00:00,0.0,0.0,0.0,0.0,0.0
2,474,2015-01-05 03:00:00,0.0,0.0,0.0,0.0,0.0
3,474,2015-01-17 09:00:00,0.0,0.0,0.0,0.0,0.0
4,26,2015-03-01 21:00:00,0.0,0.0,0.0,0.0,0.0
5,26,2015-04-15 18:00:00,0.0,0.0,0.0,0.0,0.0
6,26,2015-04-27 12:00:00,0.0,0.0,0.0,0.0,0.0
7,26,2015-05-31 15:00:00,0.0,0.0,0.0,0.0,0.0
8,26,2015-06-16 06:00:00,0.0,0.0,0.0,0.0,0.0
9,26,2015-06-27 18:00:00,0.0,0.0,0.0,0.0,0.0


## Days since last replacement from maintenance 

A crucial data set in this example is the maintenance records which contain the information of component replacement records. Possible features from this data set can be, for example, the number of replacements of each component in the last 3 months to incorporate the frequency of replacements. However, more relevent information would be to calculate how long it has been since a component is last replaced as that would be expected to correlate better with component failures since the longer a component is used, the more degradation should be expected.

As a side note, creating lagging features from maintenance data is not as straightforward as for telemetry and errors, so the features from this data are generated in a more custom way. This type of ad-hoc feature engineering is very common in predictive maintenance since domain knowledge plays a big role in understanding the predictors of a problem. In the following, the days since last component replacement are calculated for each component type as features from the maintenance data.

In [52]:
# create a column for each comp 
maint1 = maint.groupBy("machineID","datetime","comp").pivot('comp').agg(F.count('machineID').alias('dummy'))

# remove the column called comp and fill in missing values
maint2 = maint1.drop('comp').fillna(0)

# combine maintenance for a given machine in a given hour
maint_replace = (maint2.groupBy("machineID","datetime")
                 .agg(F.sum('comp1').alias('comp1sum'), 
                      F.sum('comp2').alias('comp2sum'), 
                      F.sum('comp3').alias('comp3sum'),
                      F.sum('comp4').alias('comp4sum')))

# Remove temporary dataframes
maint1.unpersist()
maint2.unpersist()

maint_replace.limit(10).toPandas().head(10)

,machineID,datetime,comp1sum,comp2sum,comp3sum,comp4sum
0,191,2015-12-28 06:00:00,1,0,0,0
1,567,2015-09-17 06:00:00,0,0,0,1
2,301,2015-04-04 06:00:00,1,1,0,0
3,852,2015-06-14 06:00:00,0,0,1,0
4,942,2015-09-23 06:00:00,0,0,1,1
5,66,2015-09-30 06:00:00,0,0,1,0
6,370,2015-07-22 06:00:00,0,1,0,1
7,512,2015-05-03 06:00:00,0,0,1,0
8,427,2015-06-15 06:00:00,0,0,1,1
9,490,2015-10-26 06:00:00,1,1,0,0


## Days since last replacement for component-1

In [53]:
# We want to align the component information on telemetry timestamps.
telemetry_times = (telemetry.withColumnRenamed('datetime','datetime_tel')
                   .drop(telemetry.volt).drop(telemetry.rotate).drop(telemetry.pressure).drop(telemetry.vibration))

# Get the counts of comp1 replacement.
maint_comp1 = (maint_replace.where((col("comp1sum") == '1'))
               .withColumnRenamed('datetime','datetime_maint')
               .drop('comp2sum', 'comp3sum', 'comp4sum'))

# Align on telemetry time
maint_tel_comp1 = telemetry_times.join(maint_comp1, ((telemetry_times['machineID']==maint_comp1['machineID']) 
                                                     & (telemetry_times['datetime_tel'] > maint_comp1['datetime_maint']) 
                                                     & (maint_comp1['comp1sum'] == '1'))).drop(maint_comp1.machineID)

# Calculate the days since comp1 replacement.
comp1 = (maint_tel_comp1.withColumn("sincelastcomp1", 
                                    datediff(maint_tel_comp1.datetime_tel, maint_tel_comp1.datetime_maint))
         .drop(maint_tel_comp1.datetime_maint).drop(maint_tel_comp1.comp1sum))

# Remove temporary dataframes
maint_tel_comp1.unpersist()
maint_comp1.unpersist()

comp1.limit(5).toPandas().head(5)

,datetime_tel,machineID,sincelastcomp1
0,2015-01-08 10:00:00,501,6
1,2015-01-08 10:00:00,501,56
2,2015-01-08 11:00:00,501,6
3,2015-01-08 11:00:00,501,56
4,2015-01-08 12:00:00,501,6
5,2015-01-08 12:00:00,501,56
6,2015-01-08 13:00:00,501,6
7,2015-01-08 13:00:00,501,56
8,2015-01-08 14:00:00,501,6
9,2015-01-08 14:00:00,501,56


## Days since last replacement for component-2

In [54]:
maint_comp2 = (maint_replace.where(col("comp2sum") == '1').withColumnRenamed('datetime','datetime_maint')
               .drop('comp1sum', 'comp3sum', 'comp4sum'))

maint_tel_comp2 = (telemetry_times.join(maint_comp2, 
                                        ((telemetry_times ['machineID']== maint_comp2['machineID']) 
                                         & (telemetry_times ['datetime_tel'] > maint_comp2['datetime_maint']) 
                                         & ( maint_comp2['comp2sum'] == '1')))
                   .drop(maint_comp2.machineID))

comp2 = (maint_tel_comp2.withColumn("sincelastcomp2", 
                                    datediff(maint_tel_comp2.datetime_tel, maint_tel_comp2.datetime_maint))
         .drop(maint_tel_comp2.datetime_maint).drop(maint_tel_comp2.comp2sum))

# Remove temporary dataframes
maint_tel_comp2.unpersist()
maint_comp2.unpersist()

comp2.limit(5).toPandas().head(5)

,datetime_tel,machineID,sincelastcomp2
0,2015-01-08 10:00:00,501,41
1,2015-01-08 11:00:00,501,41
2,2015-01-08 12:00:00,501,41
3,2015-01-08 13:00:00,501,41
4,2015-01-08 14:00:00,501,41
5,2015-01-08 15:00:00,501,41
6,2015-01-08 16:00:00,501,41
7,2015-01-08 17:00:00,501,41
8,2015-01-08 18:00:00,501,41
9,2015-01-08 19:00:00,501,41


## Days since last replacement for component-3

In [55]:
maint_comp3 = (maint_replace.where(col("comp3sum") == '1').withColumnRenamed('datetime','datetime_maint')
               .drop('comp1sum', 'comp2sum', 'comp4sum'))

maint_tel_comp3 = (telemetry_times.join(maint_comp3, ((telemetry_times ['machineID']==maint_comp3['machineID']) 
                                                      & (telemetry_times ['datetime_tel'] > maint_comp3['datetime_maint']) 
                                                      & ( maint_comp3['comp3sum'] == '1')))
                   .drop(maint_comp3.machineID))

comp3 = (maint_tel_comp3.withColumn("sincelastcomp3", 
                                    datediff(maint_tel_comp3.datetime_tel, maint_tel_comp3.datetime_maint))
         .drop(maint_tel_comp3.datetime_maint).drop(maint_tel_comp3.comp3sum))

# Remove temporary dataframes
maint_tel_comp3.unpersist()
maint_comp3.unpersist()

comp3.limit(5).toPandas().head(5)

,datetime_tel,machineID,sincelastcomp3
0,2015-01-08 10:00:00,501,206
1,2015-01-08 11:00:00,501,206
2,2015-01-08 12:00:00,501,206
3,2015-01-08 13:00:00,501,206
4,2015-01-08 14:00:00,501,206
5,2015-01-08 15:00:00,501,206
6,2015-01-08 16:00:00,501,206
7,2015-01-08 17:00:00,501,206
8,2015-01-08 18:00:00,501,206
9,2015-01-08 19:00:00,501,206


## Days since last replacement for component-4

In [56]:
maint_comp4 = (maint_replace.where(col("comp4sum") == '1').withColumnRenamed('datetime','datetime_maint')
               .drop('comp1sum', 'comp2sum', 'comp3sum'))

maint_tel_comp4 = telemetry_times.join(maint_comp4, ((telemetry_times['machineID']==maint_comp4['machineID']) 
                                                     & (telemetry_times['datetime_tel'] > maint_comp4['datetime_maint']) 
                                                     & (maint_comp4['comp4sum'] == '1'))).drop(maint_comp4.machineID)

comp4 = (maint_tel_comp4.withColumn("sincelastcomp4", 
                                    datediff(maint_tel_comp4.datetime_tel, maint_tel_comp4.datetime_maint))
         .drop(maint_tel_comp4.datetime_maint).drop(maint_tel_comp4.comp4sum))

# Remove temporary dataframes
maint_tel_comp4.unpersist()
maint_comp4.unpersist()

comp4.limit(20).toPandas().head(20)

,datetime_tel,machineID,sincelastcomp4
0,2015-01-08 10:00:00,501,131
1,2015-01-08 11:00:00,501,131
2,2015-01-08 12:00:00,501,131
3,2015-01-08 13:00:00,501,131
4,2015-01-08 14:00:00,501,131
5,2015-01-08 15:00:00,501,131
6,2015-01-08 16:00:00,501,131
7,2015-01-08 17:00:00,501,131
8,2015-01-08 18:00:00,501,131
9,2015-01-08 19:00:00,501,131


##  Combine comp1, comp2, comp3, comp4 to generate the maintenance feature set

In [57]:
# left join comp1 with (comp2, comp3, comp4) 
# left join comp2 with (comp3, comp4) 
# left join comp3, comp4 
comp3_4 = (comp3.join(comp4, ((comp3['machineID'] == comp4['machineID']) 
                              & (comp3['datetime_tel'] == comp4['datetime_tel'])), "left")
           .drop(comp4.machineID).drop(comp4.datetime_tel))

comp2_3_4 = (comp2.join(comp3_4, ((comp2['machineID'] == comp3_4['machineID']) 
                                  & (comp2['datetime_tel'] == comp3_4['datetime_tel'])), "left")
             .drop(comp3_4.machineID).drop(comp3_4.datetime_tel))

comp1_2_3_4 = (comp1.join(comp2_3_4, ((comp1['machineID'] == comp2_3_4['machineID']) 
                                      & (comp1['datetime_tel'] == comp2_3_4['datetime_tel'])), "left")
               .drop(comp2_3_4.machineID).drop(comp2_3_4.datetime_tel))

comp1_2_3_4_final = (comp1_2_3_4.groupBy("machineID", "datetime_tel")
                     .agg(F.max('sincelastcomp1').alias('sincelastcomp1'), 
                          F.max('sincelastcomp2').alias('sincelastcomp2'), 
                          F.max('sincelastcomp3').alias('sincelastcomp3'), 
                          F.max('sincelastcomp4').alias('sincelastcomp4')))

# fill in missing value
maint_count = comp1_2_3_4_final.fillna(0)

# Remove temporary dataframes
comp1_2_3_4_final.unpersist()
comp1_2_3_4.unpersist()
comp2_3_4.unpersist()
comp3_4.unpersist()
comp1.unpersist()
comp2.unpersist()
comp3.unpersist()
comp4.unpersist()
maint_replace.unpersist()

# maint_count1 maintenance 
dt_truncated = ((round(unix_timestamp(col("datetime_tel").cast("timestamp")) / time_val) * time_val)
                .cast("timestamp"))

maint_resampled = maint_count.withColumn("dt_truncated", dt_truncated)
maint_feat = (maint_resampled.groupBy("machineID","dt_truncated")
              .agg(F.mean('sincelastcomp1').alias('comp1sum'), 
                   F.mean('sincelastcomp2').alias('comp2sum'), 
                   F.mean('sincelastcomp3').alias('comp3sum'), 
                   F.mean('sincelastcomp4').alias('comp4sum')))

maint_resampled.unpersist()

maint_feat.limit(10).toPandas().head(10)

,machineID,dt_truncated,comp1sum,comp2sum,comp3sum,comp4sum
0,6,2015-11-29 18:00:00,366.000000,381.000000,471.000000,501.000000
1,7,2015-06-10 18:00:00,269.000000,224.000000,344.000000,269.000000
2,7,2015-12-13 18:00:00,455.000000,410.000000,530.000000,455.000000
3,12,2015-01-01 06:00:00,64.000000,34.000000,64.000000,79.000000
4,20,2015-10-15 03:00:00,351.000000,426.000000,456.000000,441.000000
5,29,2015-04-24 15:00:00,282.000000,117.000000,237.000000,147.000000
6,41,2015-05-26 15:00:00,359.000000,224.000000,314.000000,329.000000
7,45,2015-07-03 15:00:00,292.000000,277.000000,202.000000,397.000000
8,52,2015-06-26 00:00:00,209.666667,239.666667,359.666667,389.666667
9,53,2015-04-18 12:00:00,306.000000,321.000000,186.000000,216.000000


## Machine features

The machine features can be used without further modification. These include descriptive information about the type of each machine and its age (number of years in service). If the age information had been recorded as a "first use date" for each machine, a transformation would have been necessary to turn those into a numeric values indicating the years in service.

We do need to create a set of dummy features, boolean variables to indicate the model name of the machine. This is a _one-hot encoding_ step. 

In [58]:
# one hot encoding of the variable model
catVarNames = ['model']  
sIndexers = [StringIndexer(inputCol=x, outputCol=x + '_indexed') for x in catVarNames]
machines_cat = Pipeline(stages=sIndexers).fit(machines).transform(machines)

# one-hot encode
ohEncoders = [OneHotEncoder(inputCol=x + '_indexed', outputCol=x + '_encoded')
              for x in catVarNames]

ohPipelineModel = Pipeline(stages=ohEncoders).fit(machines_cat)
machines_cat = ohPipelineModel.transform(machines_cat)

drop_list = [col_n for col_n in machines_cat.columns if 'indexed' in col_n]

machines_feat = machines_cat.select([column for column in machines_cat.columns if column not in drop_list])

machines_feat.limit(10).toPandas().head(10)

,machineID,model,age,model_encoded
0,1,model2,18,"(0.0, 0.0, 1.0)"
1,2,model4,7,"(0.0, 1.0, 0.0)"
2,3,model3,8,"(1.0, 0.0, 0.0)"
3,4,model3,7,"(1.0, 0.0, 0.0)"
4,5,model2,2,"(0.0, 0.0, 1.0)"
5,6,model3,7,"(1.0, 0.0, 0.0)"
6,7,model4,20,"(0.0, 1.0, 0.0)"
7,8,model3,16,"(1.0, 0.0, 0.0)"
8,9,model1,7,"(0.0, 0.0, 0.0)"
9,10,model1,10,"(0.0, 0.0, 0.0)"


## Join data into feature engineering set


In [59]:
# join error features with component maintenance features
error_maint = (err_feat.join(maint_feat, 
                             ((err_feat['machineID'] == maint_feat['machineID']) 
                              & (err_feat['dt_truncated'] == maint_feat['dt_truncated'])), "left")
               .drop(maint_feat.machineID).drop(maint_feat.dt_truncated))

# now join with machines features
err_maint_mach = (error_maint.join(machines_feat, 
                                   ((error_maint['machineID'] == machines_feat['machineID'])), "left")
                  .drop(machines_feat.machineID))


err_maint_mach_feat = (err_maint_mach.select([c for c in err_maint_mach.columns if c not in 
                                              {'error1sum', 'error2sum', 'error3sum', 'error4sum', 'error5sum'}]))

# join telemetry_all with err_maint_mach_select to create final feature matrix
final_feat = (telemetry_feat.join(err_maint_mach_feat, 
                                  ((telemetry_feat['machineID'] == err_maint_mach_feat['machineID']) 
                                   & (telemetry_feat['dt_truncated'] == err_maint_mach_feat['dt_truncated'])), "left")
              .drop(err_maint_mach_feat.machineID).drop(err_maint_mach_feat.dt_truncated))

err_feat.unpersist()
maint_feat.unpersist()
machines_feat.unpersist()
telemetry_feat.unpersist()
error_maint.unpersist()
err_maint_mach.unpersist()
err_maint_mach_feat.unpersist()

final_feat.limit(10).toPandas().head(10)

,machineID,dt_truncated,volt_rollingmean_3,rotate_rollingmean_3,pressure_rollingmean_3,vibration_rollingmean_3,volt_rollingmean_24,rotate_rollingmean_24,pressure_rollingmean_24,vibration_rollingmean_24,...,error3sum_rollingmean_24,error4sum_rollingmean_24,error5sum_rollingmean_24,comp1sum,comp2sum,comp3sum,comp4sum,model,age,model_encoded
0,1,2015-01-09 06:00:00,161.758220,460.191228,96.335812,39.051541,168.100278,438.408323,100.435705,41.143014,...,0.0,0.0,0.0,117.000000,117.000000,57.000000,192.000000,model2,18,"(0.0, 0.0, 1.0)"
1,1,2015-04-02 12:00:00,177.068429,433.378560,96.498988,38.821066,172.384126,447.099119,100.856740,39.575157,...,0.0,0.0,0.0,200.000000,200.000000,140.000000,275.000000,model2,18,"(0.0, 0.0, 1.0)"
2,1,2015-04-11 18:00:00,171.014696,411.497993,97.440480,41.011138,172.095467,443.324774,98.037775,39.499885,...,0.0,0.0,0.0,209.000000,209.000000,149.000000,284.000000,model2,18,"(0.0, 0.0, 1.0)"
3,1,2015-04-17 15:00:00,161.944471,429.707358,100.453624,43.356965,166.049779,442.574675,98.379748,41.186337,...,0.0,0.0,0.0,215.000000,215.000000,155.000000,290.000000,model2,18,"(0.0, 0.0, 1.0)"
4,1,2015-04-18 21:00:00,178.593778,457.468729,101.436350,43.940496,172.648494,447.924559,99.065954,40.058577,...,0.0,0.0,0.0,216.000000,216.000000,156.000000,291.000000,model2,18,"(0.0, 0.0, 1.0)"
5,1,2015-04-22 18:00:00,173.814460,481.608946,106.691878,38.773940,170.510248,458.411583,101.900838,41.187209,...,0.0,0.0,0.0,220.000000,220.000000,160.000000,295.000000,model2,18,"(0.0, 0.0, 1.0)"
6,1,2015-05-12 00:00:00,177.166806,431.077395,98.427573,32.993633,171.314120,456.254551,101.924653,40.758712,...,0.0,0.0,0.0,239.666667,239.666667,179.666667,314.666667,model2,18,"(0.0, 0.0, 1.0)"
7,1,2015-06-02 03:00:00,169.390774,421.496917,111.307572,41.276803,165.348173,447.698849,100.007553,39.866762,...,0.0,0.0,0.0,261.000000,261.000000,201.000000,336.000000,model2,18,"(0.0, 0.0, 1.0)"
8,1,2015-07-26 12:00:00,165.344056,442.703231,87.036960,40.620244,168.613437,445.626234,96.981386,39.743980,...,0.0,0.0,0.0,315.000000,315.000000,255.000000,390.000000,model2,18,"(0.0, 0.0, 1.0)"
9,1,2015-08-04 00:00:00,165.434594,492.411368,102.873017,39.090342,171.040596,474.751091,102.689331,42.860270,...,0.0,0.0,0.0,323.666667,323.666667,263.666667,398.666667,model2,18,"(0.0, 0.0, 1.0)"


# Label construction

When using multi-class classification for predicting failure due to a problem, labelling is done by taking a time window prior to the failure of an asset and labelling the feature records that fall into that window as "about to fail due to a problem" while labelling all other records as "Â€Âœnormal." This time window should be picked according to the business case: in some situations it may be enough to predict failures hours in advance, while in others days or weeks may be needed to allow e.g. for arrival of replacement parts.

The prediction problem for this example scenerio is to estimate the probability that a machine will fail in the near future due to a failure of a certain component. More specifically, the goal is to compute the probability that a machine will fail in the next 24 hours due to a certain component failure (component 1, 2, 3, or 4). Below, a categorical failure feature is created to serve as the label. All records within a 24 hour window before a failure of component 1 have failure=comp1, and so on for components 2, 3, and 4; all records not within 24 hours of a component failure have failure=none.

In [60]:
# check to see if there are duplicate rows based on machine, datetime
failures1 = failures.dropDuplicates(['machineID', 'datetime'])

# map the failure data to final feature matrix
labeled_features = (final_feat.join(failures1, ((final_feat['machineID'] == failures1['machineID']) 
                                                & (final_feat['dt_truncated'] == failures1['datetime'])), "left")
                    .drop(failures1.machineID).drop(failures1.datetime))

# recoding the column 'failure' to be numeric double for the pyspark classification models
labeled_features1 = (labeled_features
                     .withColumn('failure', F.when(col('failure') == "comp1", 1.0).otherwise(col('failure')))
                     .withColumn('failure', F.when(col('failure') == "comp2", 2.0).otherwise(col('failure')))
                     .withColumn('failure', F.when(col('failure') == "comp3", 3.0).otherwise(col('failure')))
                     .withColumn('failure', F.when(col('failure') == "comp4", 4.0).otherwise(col('failure'))))

labeled_features2 = labeled_features1.withColumn("failure1", labeled_features1["failure"].cast(DoubleType()))
labeled_feat = labeled_features2.drop('failure').fillna(0)

failures1.unpersist()
labeled_features2.unpersist()
labeled_features1.unpersist()
final_feat.unpersist()

labeled_feat.limit(10).toPandas().head(10)

,machineID,dt_truncated,volt_rollingmean_3,rotate_rollingmean_3,pressure_rollingmean_3,vibration_rollingmean_3,volt_rollingmean_24,rotate_rollingmean_24,pressure_rollingmean_24,vibration_rollingmean_24,...,error4sum_rollingmean_24,error5sum_rollingmean_24,comp1sum,comp2sum,comp3sum,comp4sum,model,age,model_encoded,failure1
0,1,2015-01-09 06:00:00,161.758220,460.191228,96.335812,39.051541,168.100278,438.408323,100.435705,41.143014,...,0.0,0.0,117.000000,117.000000,57.000000,192.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
1,1,2015-04-02 12:00:00,177.068429,433.378560,96.498988,38.821066,172.384126,447.099119,100.856740,39.575157,...,0.0,0.0,200.000000,200.000000,140.000000,275.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
2,1,2015-04-11 18:00:00,171.014696,411.497993,97.440480,41.011138,172.095467,443.324774,98.037775,39.499885,...,0.0,0.0,209.000000,209.000000,149.000000,284.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
3,1,2015-04-17 15:00:00,161.944471,429.707358,100.453624,43.356965,166.049779,442.574675,98.379748,41.186337,...,0.0,0.0,215.000000,215.000000,155.000000,290.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
4,1,2015-04-18 21:00:00,178.593778,457.468729,101.436350,43.940496,172.648494,447.924559,99.065954,40.058577,...,0.0,0.0,216.000000,216.000000,156.000000,291.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
5,1,2015-04-22 18:00:00,173.814460,481.608946,106.691878,38.773940,170.510248,458.411583,101.900838,41.187209,...,0.0,0.0,220.000000,220.000000,160.000000,295.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
6,1,2015-05-12 00:00:00,177.166806,431.077395,98.427573,32.993633,171.314120,456.254551,101.924653,40.758712,...,0.0,0.0,239.666667,239.666667,179.666667,314.666667,model2,18,"(0.0, 0.0, 1.0)",0.0
7,1,2015-06-02 03:00:00,169.390774,421.496917,111.307572,41.276803,165.348173,447.698849,100.007553,39.866762,...,0.0,0.0,261.000000,261.000000,201.000000,336.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
8,1,2015-07-26 12:00:00,165.344056,442.703231,87.036960,40.620244,168.613437,445.626234,96.981386,39.743980,...,0.0,0.0,315.000000,315.000000,255.000000,390.000000,model2,18,"(0.0, 0.0, 1.0)",0.0
9,1,2015-08-04 00:00:00,165.434594,492.411368,102.873017,39.090342,171.040596,474.751091,102.689331,42.860270,...,0.0,0.0,323.666667,323.666667,263.666667,398.666667,model2,18,"(0.0, 0.0, 1.0)",0.0


Labels are created as 7 days before the actual failure.


In [61]:
# lag values to manually backfill label (bfill =7)
my_window = Window.partitionBy('machineID').orderBy(labeled_feat.dt_truncated.desc())

labeled_feat = labeled_feat.withColumn("prev_value1", F.lag(labeled_feat.failure1).over(my_window)).fillna(0)
labeled_feat = labeled_feat.withColumn("prev_value2", F.lag(labeled_feat.prev_value1).over(my_window)).fillna(0) 
labeled_feat = labeled_feat.withColumn("prev_value3", F.lag(labeled_feat.prev_value2).over(my_window)).fillna(0) 
labeled_feat = labeled_feat.withColumn("prev_value4", F.lag(labeled_feat.prev_value3).over(my_window)).fillna(0) 
labeled_feat = labeled_feat.withColumn("prev_value5", F.lag(labeled_feat.prev_value4).over(my_window)).fillna(0) 
labeled_feat = labeled_feat.withColumn("prev_value6", F.lag(labeled_feat.prev_value5).over(my_window)).fillna(0) 
labeled_feat = labeled_feat.withColumn("prev_value7", F.lag(labeled_feat.prev_value6).over(my_window)).fillna(0)

# create the label column 
label_bfill2 = (labeled_feat.withColumn('label', labeled_feat.failure1 + labeled_feat.prev_value1 
                                        + labeled_feat.prev_value2 + labeled_feat.prev_value3 
                                        + labeled_feat.prev_value4 + labeled_feat.prev_value5 
                                        + labeled_feat.prev_value6 + labeled_feat.prev_value7))

label_bfill2 = label_bfill2.withColumn('label_e', F.when(col('label') > 4, 4.0).otherwise(col('label')))

labeled_feat = (label_bfill2.drop(label_bfill2.prev_value1).drop(label_bfill2.prev_value2)
                .drop(label_bfill2.prev_value3).drop(label_bfill2.prev_value4)
                .drop(label_bfill2.prev_value5).drop(label_bfill2.prev_value6)
                .drop(label_bfill2.prev_value7).drop(label_bfill2.label))

labeled_feat.count()
labeled_feat.limit(10).toPandas().head(10)

,machineID,dt_truncated,volt_rollingmean_3,rotate_rollingmean_3,pressure_rollingmean_3,vibration_rollingmean_3,volt_rollingmean_24,rotate_rollingmean_24,pressure_rollingmean_24,vibration_rollingmean_24,...,error5sum_rollingmean_24,comp1sum,comp2sum,comp3sum,comp4sum,model,age,model_encoded,failure1,label_e
0,26,2016-01-01 06:00:00,150.373071,468.392372,123.049312,41.064981,165.947064,458.076927,125.103807,41.257244,...,0.0,519.000000,384.000000,429.000000,534.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
1,26,2016-01-01 03:00:00,171.312845,468.860510,126.401124,44.048947,167.322752,453.389295,125.617829,41.378512,...,0.0,519.000000,384.000000,429.000000,534.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
2,26,2016-01-01 00:00:00,162.823386,481.743444,133.493631,41.836345,166.734221,453.736499,123.019190,40.938805,...,0.0,518.666667,383.666667,428.666667,533.666667,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
3,26,2015-12-31 21:00:00,164.194690,462.899569,125.765224,42.479379,167.362472,448.528873,118.071148,40.729783,...,0.0,518.000000,383.000000,428.000000,533.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
4,26,2015-12-31 18:00:00,160.712718,437.101736,120.259696,41.486162,167.722067,447.790716,114.343428,40.670699,...,0.0,518.000000,383.000000,428.000000,533.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
5,26,2015-12-31 15:00:00,166.736647,439.093650,127.331781,39.365970,168.004840,449.484417,111.482501,40.522237,...,0.0,518.000000,383.000000,428.000000,533.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
6,26,2015-12-31 12:00:00,179.352011,462.247082,118.797268,37.398279,167.438445,446.129050,107.663676,41.166588,...,0.0,518.000000,383.000000,428.000000,533.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
7,26,2015-12-31 09:00:00,172.138343,451.637524,125.204416,42.720838,166.867088,441.486441,105.177142,41.715829,...,0.0,518.000000,383.000000,428.000000,533.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
8,26,2015-12-31 06:00:00,161.311373,423.530842,127.689489,41.692179,167.794154,438.170394,102.654382,41.259651,...,0.0,518.000000,383.000000,428.000000,533.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0
9,26,2015-12-31 03:00:00,166.604600,471.638142,105.612012,40.531290,168.546591,437.765909,97.494040,41.387190,...,0.0,518.000000,383.000000,428.000000,533.000000,model3,3,"(1.0, 0.0, 0.0)",0.0,0.0


## Write the feature data to blob

In [ ]:
# write the final result as parquet file in blob location 
# https://github.com/Azure/ViennaDocs/blob/master/Documentation/UsingBlobForStorage.md

# Create a new container if necessary, otherwise you can use an existing container.
# This command creates the container if it does not already exist. Else it does nothing.
my_service.create_container(STORAGE_CONTAINER_NAME, 
                            fail_on_exist=False, 
                            public_access=PublicAccess.Container)

# you decide to partition the dataframe into three files and save them in the current folder.
# if you wish to visualize them in the run history Output Files, specify the path 
# as './outputs/multiple_files.parquet'.
#label_bfill3.coalesce(3).write.mode('overwrite').parquet('multiple_files.parquet')
labeled_feat.write.mode('overwrite').parquet(FEATURES_LOCAL_DIRECT)

# unlike the single file case, for multiple files we need to first delete results from the 
# previous run before uploading.
for blob in my_service.list_blobs(STORAGE_CONTAINER_NAME):
    if FEATURES_LOCAL_DIRECT in blob.name:
        my_service.delete_blob(STORAGE_CONTAINER_NAME, blob.name)

# upload the entire folder into blob storage
for name in glob.iglob(FEATURES_LOCAL_DIRECT + '/*'):
    print(os.path.abspath(name))
    my_service.create_blob_from_path(STORAGE_CONTAINER_NAME, name, name)

print("Feature engineering final dataset files saved!")

# Conclusion
